In [ ]:
#install tensorflow and keras
!pip uninstall tensorflow tensorflow-gpu tensorflow-text protocol --yes
!pip install tensorflow==2.17.0
!pip install tensorflow-text==2.17.0
!pip install tensorflow-gpu==2.12.0

!pip uninstall keras --yes
!pip install --upgrade keras-nlp
!pip install --upgrade keras
!pip install --upgrade keras-nlp-nightly


Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
Found existing installation: tensorflow-text 2.17.0
Uninstalling tensorflow-text-2.17.0:
  Successfully uninstalled tensorflow-text-2.17.0
  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.3 MB)
  Using cached tensorflow_text-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.8 kB)
Using cached tensorflow_text-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (s

In [ ]:
#install kgptalkie
!pip uninstall openai --yes
!pip uninstall langsmith --yes
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git --upgrade --force-reinstall
!pip install googletrans


  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-rl997_r1
  Running command git clone --filter=blob:none --quiet https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-rl997_r1
  Resolved https://github.com/laxmimerit/preprocess_kgptalkie.git to commit 0f56a6502679f6750b7f0ffed98f590d1f0246bd
  Preparing metadata (setup.py) ... done
  Created wheel for preprocess_kgptalkie: filename=preprocess_kgptalkie-0.11-py3-none-any.whl size=8175 sha256=10c187f94d6353b5286cef1607365597394aa4556beb2f5cf14430a2b7a9a538
  Stored in directory: /tmp/pip-ephem-wheel-cache-8iinfoay/wheels/5c/94/34/99d5ff65e88b8d9a6c5e8d8652f2311d87790a61a1b7466e21
Successfully built preprocess_kgptalkie
  Attempting uninstall: preprocess_kgptalkie
    Found existing installation: preprocess_kgptalkie 0.11
    Uninstalling preprocess_kgptalkie-0.11:
      Successfully uninstalled preprocess_kgptalkie-0.11


In [ ]:
#python modules imports
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

import numpy as np
import pandas as pd
import tensorflow as tf
#import keras_core as keras
import keras
import keras_nlp
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import preprocess_kgptalkie as kgp

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("KerasNLP version:", keras_nlp.__version__)


TensorFlow version: 2.17.0
Keras version: 3.7.0
KerasNLP version: 0.19.0.dev202412160356


In [ ]:
#import data
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
print("Training dim = {} memory = {:.2f} MB".format(df_train.shape, df_train.memory_usage().sum() / 1024 ** 2))
print("Test dim = {} memory = {:.2f} MB".format(df_test.shape, df_test.memory_usage().sum() / 1024 ** 2))

df_train["length"] = df_train["text"].apply(lambda x : len(x))
df_test["length"] = df_test["text"].apply(lambda x : len(x))

print("Train Length Stat")
print(df_train["length"].describe())

print("Test Length Stat")
print(df_test["length"].describe())

df_train.head()
df_test.head()


Training dim = (7613, 5) memory = 0.29 MB
Test dim = (3263, 4) memory = 0.10 MB
Train Length Stat
count    7613.000000
mean      101.037436
std        33.781325
min         7.000000
25%        78.000000
50%       107.000000
75%       133.000000
max       157.000000
Name: length, dtype: float64
Test Length Stat
count    3263.000000
mean      102.108183
std        33.972158
min         5.000000
25%        78.000000
50%       109.000000
75%       134.000000
max       151.000000
Name: length, dtype: float64


,id,keyword,location,text,length
0,0,NaN,NaN,Just happened a terrible car crash,34
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45


In [ ]:
#data cleaning
def get_clean(x):
  x = str(x).lower().replace("\\","  ").replace("_", " ").replace(".", " ")
  #x = kgp.cont_exp(x)
  x = kgp.remove_emails(x)
  x = kgp.remove_urls(x)
  x = kgp.remove_html_tags(x)
  x = kgp.remove_rt(x)
  x = kgp.remove_accented_chars(x)
  x = kgp.remove_special_chars(x)
  #x = kgp.remove_dups_char(x)
  return x

df_train["text"] = df_train["text"].apply(lambda x: get_clean(x))
df_test["text"] = df_test["text"].apply(lambda x: get_clean(x))

df_train.head()
df_test.head()


/usr/local/lib/python3.10/dist-packages/preprocess_kgptalkie/text_preprocess.py:92: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(x, 'lxml').get_text()


,id,keyword,location,text,length
0,0,NaN,NaN,just happened a terrible car crash,34
1,2,NaN,NaN,heard about earthquake is different cities sta...,64
2,3,NaN,NaN,there is a forest fire at spot pond geese are ...,96
3,9,NaN,NaN,apocalypse lighting spokane wildfires,40
4,11,NaN,NaN,typhoon soudelor kills 28 in china and taiwan,45


In [6]:
#training with DistilBERT model
BATCH_SIZE = 32
NUM_TRAINING_EXAMPLES = df_train.shape[0]
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.2
STEPS_PER_EPOCH = int(NUM_TRAINING_EXAMPLES) * TRAIN_SPLIT // BATCH_SIZE
EPOCHS = 1
AUTO = tf.data.experimental.AUTOTUNE

X = df_train["text"]
y = df_train["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VAL_SPLIT, random_state=42)
X_test = df_test["text"]

preset = "distil_bert_base_en_uncased"

preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(preset, sequence_length=160, name="processor_4_tweets")

classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset, preprocessor = preprocessor, num_classes=2)

classifier.summary()

classifier.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

history = classifier.fit(x = X_train, y = y_train, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = (X_val, y_val))


Preprocessor: "processor_4_tweets"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ distil_bert_tokenizer (DistilBertTokenizer)                   │                       Vocab size: 30,522 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "distil_bert_text_classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ distil_bert_backbone          │ (None, None, 768)         │      66,362,880 │ padding_mask[0][0],        │
│ (DistilBertBackbone)          │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ get_item (GetItem)            │ (None, 768)               │               0 │ distil_bert_backbone[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pooled_dense (Dense)          │ (None, 768)               │         590,592 │ get_item[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output_dropout (Dropout)      │ (None, 768)               │               0 │ pooled_dense[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ logits (Dense)                │ (None, 2)                 │           1,538 │ output_dropout[0][0]       │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 66,955,010 (255.41 MB)

 Trainable params: 66,955,010 (255.41 MB)

 Non-trainable params: 0 (0.00 B)

191/191 ━━━━━━━━━━━━━━━━━━━━ 9333s 49s/step - accuracy: 0.5309 - loss: 1.2275 - val_accuracy: 0.5739 - val_loss: 0.6931


In [ ]:
#inference with DistilBERT model
y_pred_train = classifier.predict(X_train)


In [11]:
#f1-score with DistilBERT model
print(y_pred_train)


[[-4.932431  -5.504407 ]
 [-4.9322476 -5.504804 ]
 [-4.932391  -5.5044785]
 ...
 [-4.932665  -5.5039606]
 [-4.932289  -5.5047297]
 [-4.9322796 -5.504747 ]]


In [19]:
#training with CNN model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D

#tokenize text
text = df_train["text"]
token = Tokenizer()
token.fit_on_texts(text)
vocab_size = len(token.word_index) + 1
encoded_text = token.texts_to_sequences(text) #turn words into indexed tokens
print("encoded_text:", encoded_text)
max_length = 40
X = pad_sequences(encoded_text, maxlen=max_length, padding="post")
print("X:", X)
y = df_train["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

#construct CNN network
vec_size = 100
model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length))

model.add(Conv1D(32, 1, activation="relu"))
model.add(MaxPooling1D(2))
model.add(Dropout(0.5))

model.add(Dense(32, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(16, activation="relu"))

model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation="sigmoid"))

model.summary()

#fitting the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


encoded_text: [[111, 4564, 21, 1, 839, 5, 18, 240, 125, 1576, 4565, 81, 38], [177, 40, 218, 771, 6875, 6876, 1353], [38, 1707, 1577, 4, 2097, 3, 661, 21, 122, 6877, 17, 1708, 39, 420, 241, 52, 2097, 3, 661, 1354, 21, 1068], [2851, 55, 4566, 1355, 241, 1354, 3, 87], [29, 92, 1176, 18, 316, 19, 6878, 2098, 25, 254, 19, 1355, 6879, 66, 2, 178], [2852, 374, 87, 1578, 921, 840, 3, 642, 6880, 538, 4, 1124, 375, 40, 4567, 1355], [212, 72, 841, 278, 1177, 800, 242, 5, 1709, 3, 6881, 1010, 2384, 1579], [31, 10, 219, 5, 1, 1889, 6, 7, 70, 103, 2, 40, 3, 1, 4568], [279, 41, 71, 241, 1263, 47, 3, 1, 575, 842, 1, 711], [31, 2385, 15, 1, 490, 8, 235, 4, 111, 340], [600, 55, 617, 19, 1, 270, 471, 30, 618], [843, 619, 2853, 8, 213, 2854, 3520, 662, 2, 663, 7, 193, 3, 619, 2853, 54, 161, 7, 301, 67, 54, 161, 7, 301, 67, 6882, 242], [2855, 242, 2099, 6883, 2853, 1356, 52, 2386, 620, 302, 739, 3521], [212, 3, 4569, 922, 45, 1710, 4569], [227, 4, 178, 393, 10, 3522, 3, 4570, 123, 83, 356], [601, 28, 98], 

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ ?                           │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - accuracy: 0.5719 - loss: 0.6784 - val_accuracy: 0.6999 - val_loss: 0.6319
Epoch 2/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 8s 44ms/step - accuracy: 0.7201 - loss: 0.5951 - val_accuracy: 0.7722 - val_loss: 0.5187
Epoch 3/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.8311 - loss: 0.4351 - val_accuracy: 0.7958 - val_loss: 0.4856
Epoch 4/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - accuracy: 0.8737 - loss: 0.3241 - val_accuracy: 0.7820 - val_loss: 0.4879
Epoch 5/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.9086 - loss: 0.2424 - val_accuracy: 0.7741 - val_loss: 0.5226


In [24]:
#inference with CNN model
def get_encoded(x):
  x = get_clean(x)
  x = token.texts_to_sequences([x])
  x = pad_sequences(x, maxlen=max_length, padding="post")
  return x

y_pred_test = model.predict(X_test)
y_pred_test = y_pred_test > 0.4
X_val = get_encoded(X_test)
print("y_pred_test:", y_pred_test)
print("y_test:", y_test)

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
y_pred_test: [[False]
 [False]
 [False]
 ...
 [ True]
 [ True]
 [ True]]
y_test: 3311    0
541     0
2687    1
495     1
2775    1
       ..
3822    0
375     1
2137    0
3108    1
2168    1
Name: target, Length: 1523, dtype: int64


In [ ]:
#f1-score with CNN model
